In [7]:
#Random Forest algorithm
#replicate findings from Walters et al 2014

In [2]:
#import io, os, sys, types
#from IPython import get_ipython
#from nbformat import read
#from IPython.core.interactiveshell import InteractiveShell

import biom
import sklearn
import pandas as pd

In [3]:
#HMP dataset
text_file = "/Users/shwetakinger/Desktop/ERSP_data/merged_bmi_mapping_final__original_study_HMP__.txt"
biom_file = "/Users/shwetakinger/Downloads/per_study_otu_tables/filtered_otu_table__original_study_HMP__.biom"

In [4]:
m = pd.read_csv(text_file, sep="\t", index_col=0)

In [5]:
m.columns

Index(['BarcodeSequence', 'LinkerPrimerSequence', 'BMI', 'bmi_group_binned',
       'bmi_group_coded', 'original_study', 'combined_study_bmi_group',
       'PCR_PRIMERS', 'TARGET_SUBFRAGMENT', 'AGE', 'ELEVATION', 'LONGITUDE',
       'COUNTRY', 'SEQUENCING_METH', 'SAMPLE_CENTER', 'Description_duplicate',
       'ReversePrimer', 'COLLECTION_DATE', 'SEX', 'FAMILY_RELATIONSHIP_GG',
       'STUDY_CENTER', 'EXPERIMENT_CENTER', 'bmi_group_amish', 'RUN_CENTER',
       'LATITUDE', 'Description'],
      dtype='object')

In [6]:
m.index

Index(['SRS011271.396367', 'SRS011410.396107', 'SRS011415.394388',
       'SRS011529.396185', 'SRS011586.393765', 'SRS011621.394454',
       'SRS012191.395151', 'SRS013177.401537', 'SRS013386.400409',
       'SRS013543.394407',
       ...
       'SRS049157.403739', 'SRS049959.401022', 'SRS056620.394943',
       'SRS057447.396161', 'SRS063068.403097', 'SRS063524.397569',
       'SRS063827.396534', 'SRS065263.395822', 'SRS065466.395776',
       'SRS065725.395071'],
      dtype='object', name='#SampleID', length=208)

In [7]:
b= biom.load_table(biom_file)

In [ ]:
#select only samples that fit LEAN OR OBESE
i = m[m.bmi_group_coded != "Overweight"].index
#print(i)
i_2 = m[m.bmi_group_coded != "Overweight"]
print(i_2)
i_2.loc[i_2.bmi_group_binned == 3, 'bmi_group_binned'] = 0
print(i_2.bmi_group_binned)

In [18]:
writer = pd.ExcelWriter('biom_original_HMP.xlsx', engine='xlsxwriter')

In [8]:
#get biom file
p = pd.DataFrame(b.matrix_data.T.todense().astype(int), index=b.ids(axis="sample"), columns=b.ids(axis="observation"))
#print(p)

In [20]:
# Convert the dataframe to an XlsxWriter Excel object.
p.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [10]:
# load sorted file
sorted_m = pd.read_csv("/Users/shwetakinger/Desktop/biom_final_HMP.csv", index_col=0)
#print(sorted_m)

In [18]:
data = pd.DataFrame([])
for n in range(0, 143):
    #print(n)
    num = sorted_m.index.get_loc(i[n])
    data = data.append(sorted_m.iloc[[num]])
print(data)

                  304309  193008  190299  191393  193061  186596  174924  \
SRS011271.396367       0       0       0       0       0       0       0   
SRS011410.396107       0       0       0       0       0       0       2   
SRS011415.394388       0       0       0       0       0       0       0   
SRS011529.396185       0       0       0       0       0       0       0   
SRS011586.393765       0       0       0       0       0       0       0   
SRS011621.394454       0       0       0       0       0       0       0   
SRS012191.395151       0       0       0       0       0       0       1   
SRS013177.401537       0       0       0       0       0       0       0   
SRS013386.400409       0       0       0       0       0       0       0   
SRS013543.394407       0       0       0       0       0       0       1   
SRS013687.400715       0       0       0       0       0       0       0   
SRS013801.398248       0       0       0       0       0       0       0   
SRS014345.39

In [19]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier


In [ ]:
from sklearn.cross_validation import cross_val_score
iter_range = range(1, 6)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=2000)
    #cross-validation AUC score
    scores = cross_val_score(classifier, data, i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    rForest_scores.append(scores.mean())
    average_accuracy+=scores.mean()

print(rForest_scores)
print(average_accuracy/(5.0))